# Рекурентные сети для обработки последовательностей

Вспомним все, что мы уже знаем про обработку текстов:
- Компьютер не понимает текст, поэтому нам нужно его как-то закодировать - представить в виде вектора
- В тексте много повторяющихся слов/лишний слов - нужно сделать препроцессинг:
    - удалить знаки препинания
    - удалить стоп-слова
    - привести слова к начальной форме (**стемминг** и **лемматизация**)
    - ???
    
    
- После этого мы можем представить наш текст (набор слов) в виде вектора, например, стандартными способами:
    - **CounterEncoding** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=1,\ если\ vocab[i]\ \in\ doc$
    - **HashingVectorizer** - вектор заранее заданной длины
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=1,\ если\ \exists\ txt\ \in\ doc:\ hash(text)\ =\ i$
    - **TfIdfVectorizer** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=tf(vocab[i])*idf(vocab[i]),\ если\ vocab[i]\ \in\ doc$
    
        $$ tf(t,\ d)\ =\ \frac{n_t}{\sum_kn_k} $$
        $$ idf(t,\ D)\ =\ \log\frac{|D|}{|\{d_i\ \in\ D|t\ \in\ D\}|} $$
        
, где 
- $n_t$ - число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе
- $|D|$ — число документов в коллекции;
- $|\{d_i\ \in\ D\mid\ t\in d_i\}|$— число документов из коллекции $D$, в которых встречается $t$ (когда $n_t\ \neq\ 0$).



Это база и она работает. Мы изучили более продвинутые подходы: эмбединги и сверточные сети по эмбедингам. Но тут есть проблема: любой текст - это последовательность, ни эмбединги, ни сверточные сети не работают с ним как с последовательностью. Так давайте попробуем придумать архитектуру, которая будет работать с текстом как с последовательностью, двигаясь по эмбедингам и как-то меняя их значения.

## Придумаем сами архитектуру, чтобы работать с последовательностями

Возьмем перцептрон с входом - эмбедингом слова (пусть пока он фиксированный) и будем пытаться классифицировать каждое слово.

Почему классифицировать? потому что это частая задача в обработке языка + это дает возможность генерировать текст (просто классифицируем на кол-во классов = кол-ву слов в словаре).

<img src="images/Single_layer_perceptron.png">

Какая тут последовательность? никакой, но давайте на вход подавать эмбединг, но в 1 скрытый слой будем добавлять последний скрытый слой предыдущего шага)

<img src="images/Rnnbr.png">

То есть мы прокидываем информацию с предыдущего шага, а за счет того, что мы все время так стекаем вектора мы получаем то, что информация проходит через текст от начала до конца. Что делать с 1 шагом? -> Добавим вектор из нулей. И вот мы получили первую рекурентную сеть. Чаще её рисуют следующим образом:


<img src="images/Rnn.png">

Итак, мы придумали простую рекуретную сеть. Последний открытый вопрос как её обучать?

Все также, градиентным спуском, нам нужно двигаться во времени и обновлять параметры, поэтому обучение таких сетей занимает очень много времени (вы не можете обновить веса для 1-го токена, пока не посчитаете градиент сквозь время).


Что делать, если мы хотим классифицировать текст целиком? оставить только последний выход!

<img src="images/RnnTasks.png">

In [1]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")



ModuleNotFoundError: No module named 'stop_words'

In [ ]:
df_train.head()

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

# Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

Как решить? -> LSTM


<img src="images/lstm.png">


https://colah.github.io/posts/2015-08-Understanding-LSTMs/


Давайте, кратко посмотрим как это работает:


<img src="images/LSTMMaths.png">

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

# Какие проблемы:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU


<img src="images/gru.png">


GRU Math


<img src="images/GRUMath.png">


In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

3 подхода:

<img src="images/RNNCompar.png">


Как регуляризовать?
- дропаут
- рекурентный дропаут


<img src="images/Dropouts.png">